<h1>SPEED DATING EDA</h1>

<img src="img/image.jpg" alt="Image" width="30%" height="30%">

## PARTIE 1 : Chargement des librairies et du dataset

In [13]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_colwidth', 200)

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [14]:
dataset = pd.read_csv('src/Speed_Dating_Data.csv', encoding = 'latin-1')

## PARTIE 2 : Data cleaning

In [15]:
# Choix des colonnes + renommer + drop lignes erreurs

df = dataset.loc[:,
                 ['iid', 'gender', 'condtn', 'round', 'order', 'pid', 'match', 'int_corr', 'samerace', 
                  'age_o', 'race_o', 'dec_o', 'met_o', 'age', 'field', 'undergra', 'race', 'imprace', 'imprelig', 
                  'from', 'income', 'goal', 'date', 'go_out', 'career', 'exphappy', 'expnum',
                  'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                  'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o']].copy()

df = df.iloc[:-1,:]

df.columns = ['id_c', 'genre_c', 'choix_c', 'round_c', 'ordre_p', 'id_p', 'match', 'corr_p', 'memerace_p',
              'age_p', 'race_p', 'decision_p', 'rencontre_p', 'age_c', 'etude_c', 'ecole_c', 'race_c', 'imprace_c', 'impreligion_c',
              'from_c', 'revenue_c', 'objectif_c', 'date_c', 'sortir_c', 'carriere_c', 'satisfaction_c', 'interesse_c',
              'pref_attirance_p', 'pref_sincerite_p', 'pref_intelligence_p', 'pref_fun_p', 'pref_ambition_p', 'pref_interets_p',
              'note_attirance_p', 'note_sincerite_p', 'note_intelligence_p', 'note_fun_p', 'note_ambition_p', 'note_interets_p', 'note_globale_p']

df = df.drop(df[df['id_c'].isin([58, 59, 136, 339, 340, 346])].index)

In [16]:
# Remplacer les NaN par la médiane
columns_median = ['age_c', 'imprace_c', 'impreligion_c', 'satisfaction_c', 'corr_p']
df[columns_median] = df[columns_median].apply(pd.to_numeric, errors='coerce')
median_replace = df[columns_median].mean()
df[columns_median] = df[columns_median].fillna(median_replace)

# Remplacer les NaN par le plus fréquent (.mode pour avoir le plus fréquent)
columns_mostfrequent = ['from_c', 'carriere_c', 'age_p']
modes_replace = df[columns_mostfrequent].mode().iloc[0]
df[columns_mostfrequent] = df[columns_mostfrequent].fillna(modes_replace)

# Remplacer l'id partenaire vide par un nouvel id
df['id_p'].fillna(553, inplace=True)

In [17]:
# Transformation des valeurs (int vers str)

df['genre_c'] = df['genre_c'].apply(lambda x: 'Femme' if x == 0 else 'Homme' if x == 1 else 'Autre')
df['choix_c'] = df['choix_c'].apply(lambda x: 'Choix limité' if x == 1 else 'Choix vaste' if x == 2 else 'Autre')
df['memerace_p'] = df['memerace_p'].apply(lambda x: 'Non' if x == 0 else 'Oui' if x == 1 else 'Autre')
df['rencontre_p'] = df['rencontre_p'].apply(lambda x: 'Oui' if x == 1 else 'Non' if x == 2 else 'Autre')

# df['match'] = df['match'].apply(lambda x: 'Pas match' if x == 0 else 'Match' if x == 1 else 'Autre')

for column in df[['race_p', 'race_c']]:
    df[column] = df[column].apply(lambda x: 
    'Black' if x == 1 else
    'European' if x == 2 else
    'Latino' if x == 3 else
    'Asian' if x == 4 else
    'Native American' if x == 5 else
    'Other')
    
df['objectif_c'] = df['objectif_c'].apply(lambda x: 
    'Seemed like a fun night out' if x == 1 else
    'To meet new people' if x == 2 else
    'To get a date' if x == 3 else
    'Looking for a serious relationship' if x == 4 else
    'To say I did it' if x == 5 else
    'Other')

df['date_c'] = df['date_c'].apply(lambda x: 
    'Several times a week' if x == 1 else
    'Twice a week' if x == 2 else
    'Once a week' if x == 3 else
    'Twice a month' if x == 4 else
    'Once a month' if x == 5 else
    'Several time a year' if x == 6 else
    'Almost never')

df['sortir_c'] = df['sortir_c'].apply(lambda x: 
    'Several times a week' if x == 1 else
    'Twice a week' if x == 2 else
    'Once a week' if x == 3 else
    'Twice a month' if x == 4 else
    'Once a month' if x == 5 else
    'Several time a year' if x == 6 else
    'Almost never')

## PARTIE 3 : Explication des données

In [18]:
explications = pd.read_csv('src/explications.csv')
explications

,Colonne,Description
0,id_c,Identifiant du candidat
1,genre_c,Genre du candidat
2,choix_c,Le candidat a limité son nombre de partenaires ?
3,round_c,Nombre de round que le candidat a effectué
4,ordre_p,Ordre de rencontre du partenaire
5,id_p,Identifiant du partenaire
6,match,"Second date ou non ? 1 = oui, 0 = non"
7,corr_p,Correlation entre candidat et partenaire sur les intérêts
8,memerace_p,Le partenaire est-il de la même race ?
9,age_p,Age du partenaire


In [19]:
df.head()

,id_c,genre_c,choix_c,round_c,ordre_p,id_p,match,corr_p,memerace_p,age_p,race_p,decision_p,rencontre_p,age_c,etude_c,ecole_c,race_c,imprace_c,impreligion_c,from_c,revenue_c,objectif_c,date_c,sortir_c,carriere_c,satisfaction_c,interesse_c,pref_attirance_p,pref_sincerite_p,pref_intelligence_p,pref_fun_p,pref_ambition_p,pref_interets_p,note_attirance_p,note_sincerite_p,note_intelligence_p,note_fun_p,note_ambition_p,note_interets_p,note_globale_p
0,1,Femme,Choix limité,10,4,11.0,0,0.14,Non,27.0,European,0,Non,21.0,Law,NaN,Asian,2.0,4.0,Chicago,"69,487.00",To meet new people,Almost never,Several times a week,lawyer,3.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,7.0
1,1,Femme,Choix limité,10,3,12.0,0,0.54,Non,22.0,European,0,Non,21.0,Law,NaN,Asian,2.0,4.0,Chicago,"69,487.00",To meet new people,Almost never,Several times a week,lawyer,3.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,8.0
2,1,Femme,Choix limité,10,10,13.0,1,0.16,Oui,22.0,Asian,1,Oui,21.0,Law,NaN,Asian,2.0,4.0,Chicago,"69,487.00",To meet new people,Almost never,Several times a week,lawyer,3.0,2.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
3,1,Femme,Choix limité,10,5,14.0,1,0.61,Non,23.0,European,1,Non,21.0,Law,NaN,Asian,2.0,4.0,Chicago,"69,487.00",To meet new people,Almost never,Several times a week,lawyer,3.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,7.0
4,1,Femme,Choix limité,10,7,15.0,1,0.21,Non,24.0,Latino,1,Non,21.0,Law,NaN,Asian,2.0,4.0,Chicago,"69,487.00",To meet new people,Almost never,Several times a week,lawyer,3.0,2.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,8.0


In [20]:
# Stats descriptives des colonnes

df.describe(include='all')

,id_c,genre_c,choix_c,round_c,ordre_p,id_p,match,corr_p,memerace_p,age_p,race_p,decision_p,rencontre_p,age_c,etude_c,ecole_c,race_c,imprace_c,impreligion_c,from_c,revenue_c,objectif_c,date_c,sortir_c,carriere_c,satisfaction_c,interesse_c,pref_attirance_p,pref_sincerite_p,pref_intelligence_p,pref_fun_p,pref_ambition_p,pref_interets_p,note_attirance_p,note_sincerite_p,note_intelligence_p,note_fun_p,note_ambition_p,note_interets_p,note_globale_p
count,8314.000000,8314,8314,8314.000000,8314.000000,8314.000000,8314.000000,8314.000000,8314,8314.000000,8314,8314.000000,8314,8314.000000,8314,4913,8314,8314.000000,8314.000000,8314,4279,8314,8314,8314,8314,8314.000000,1800.000000,8225.000000,8225.000000,8225.000000,8216.000000,8207.000000,8185.000000,8109.000000,8037.000000,8017.000000,7963.000000,7603.000000,7248.000000,8072.000000
unique,NaN,2,2,NaN,NaN,NaN,NaN,NaN,2,NaN,5,NaN,3,NaN,259,241,5,NaN,NaN,269,261,6,7,7,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Homme,Choix vaste,NaN,NaN,NaN,NaN,NaN,Non,NaN,European,NaN,Non,NaN,Business,UC Berkeley,European,NaN,NaN,New York,"55,080.00",Seemed like a fun night out,Several time a year,Twice a week,Finance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4173,6925,NaN,NaN,NaN,NaN,NaN,4998,NaN,4687,NaN,7584,NaN,521,107,4726,NaN,NaN,538,124,3425,2094,2990,228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,284.004931,NaN,NaN,16.909189,8.945754,284.470291,0.164301,0.196032,NaN,26.366009,NaN,0.418331,NaN,26.359092,NaN,NaN,NaN,3.785129,3.651964,NaN,NaN,NaN,NaN,NaN,NaN,5.533591,5.570556,22.476385,17.402237,20.286140,17.447517,10.688697,11.844720,6.183857,7.174319,7.368467,6.396710,6.776338,5.467784,6.129175
std,158.689998,NaN,NaN,4.338560,5.481447,158.900958,0.370571,0.301811,NaN,3.537417,NaN,0.493315,NaN,3.560075,NaN,NaN,NaN,2.842975,2.802554,NaN,NaN,NaN,NaN,NaN,NaN,1.729502,4.762569,12.558723,7.018927,6.782085,6.078848,6.096385,6.355888,1.949887,1.742394,1.551372,1.954481,1.795186,2.155159,1.840920
min,1.000000,NaN,NaN,5.000000,1.000000,1.000000,0.000000,-0.830000,NaN,18.000000,NaN,0.000000,NaN,18.000000,NaN,NaN,NaN,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,154.250000,NaN,NaN,15.000000,4.000000,155.000000,0.000000,-0.010000,NaN,24.000000,NaN,0.000000,NaN,24.000000,NaN,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,2.000000,15.000000,15.000000,17.500000,15.000000,5.000000,9.520000,5.000000,6.000000,6.000000,5.000000,6.000000,4.000000,5.000000
50%,281.000000,NaN,NaN,18.000000,8.000000,281.000000,0.000000,0.210000,NaN,26.000000,NaN,0.000000,NaN,26.000000,NaN,NaN,NaN,3.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,4.000000,20.000000,18.370000,20.000000,18.000000,10.000000,10.640000,6.000000,7.000000,7.000000,7.000000,7.000000,6.000000,6.000000
75%,408.000000,NaN,NaN,20.000000,13.000000,409.000000,0.000000,0.430000,NaN,28.000000,NaN,1.000000,NaN,28.000000,NaN,NaN,NaN,6.000000,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,7.000000,8.000000,25.000000,20.000000,23.810000,20.000000,15.000000,16.000000,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,7.000000


## PARTIE 4 : Analyse exploratoire des données

In [21]:
gender_count = df.groupby('genre_c')['id_c'].nunique()
px.pie(gender_count, 
       values = gender_count.values, 
       names = gender_count.index, 
       title = 'Répartition du genre', 
       width = 500)

Nous avons ici une répartition de quasiment 50/50 hommes/femmes. Le léger décalage (de 5 femmes en moins) est lié au data cleaning et à la suppression de quelques lignes avec trop de données manquantes.

In [22]:
px.histogram(df.drop_duplicates('id_c')['age_c'], 
             x = 'age_c', 
             title = 'Répartition des âges des participants', 
             histnorm = 'percent',
             width = 1000)

On a ici un échantillon avec une moyenne d'âge assez jeune (26 ans), expliqué par 2 aspects :

- Représente la majorité des utilisateurs de site de rencontre en général

- Les speed datings étant réalisés dans une université

In [23]:
categories = ['Attirance', 'Sincérité', 'Intelligence', 'Fun', 'Ambition', 'Intérêts']

fig = go.Figure()
fig.add_trace(go.Scatterpolar(
      r=df[df['genre_c'] == 'Femme'][['pref_attirance_p', 'pref_sincerite_p', 'pref_intelligence_p', 'pref_fun_p', 'pref_ambition_p', 'pref_interets_p']].mean(),
      theta=categories,
      fill='toself',
      name='Homme'
))
fig.add_trace(go.Scatterpolar(
      r=df[df['genre_c'] == 'Homme'][['pref_attirance_p', 'pref_sincerite_p', 'pref_intelligence_p', 'pref_fun_p', 'pref_ambition_p', 'pref_interets_p']].mean(),
      theta=categories,
      fill='toself',
      name='Femme'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=False,
      range=[0, 30],
      ),
    ),
  showlegend=True,
    polar_angularaxis=dict(
    rotation=90,
    ),
  width = 900,
  height = 600,
  title = 'Représentation des préférences chez les hommes et les femmes',
  title_font = dict(
    size=20
    ),
)

fig.show()

Pour tout le monde, les attributs intérêts et ambition semblent être ceux qui les intéressent les moins.

Les hommes semblent nettement plus intéressés par l'attirance que par les autres attributs. 

Les femme ont une répartition un peu plus homogènes entre les autres attributs, avec une légère dominante pour l'intelligence

In [30]:
corr_homme = df[df['genre_c'] == 'Homme'][['note_attirance_p', 'note_sincerite_p', 'note_intelligence_p', 'note_fun_p', 'note_ambition_p', 'note_interets_p', 'note_globale_p', 'match']].corr()
corr_femme = df[df['genre_c'] == 'Femme'][['note_attirance_p', 'note_sincerite_p', 'note_intelligence_p', 'note_fun_p', 'note_ambition_p', 'note_interets_p', 'note_globale_p', 'match']].corr()

fig = make_subplots(rows=1, cols=2, subplot_titles=['Partenaire = Femme', 'Partenaire = Homme'])

heatmap_homme = px.imshow(corr_homme, origin = 'lower')
fig.add_trace(heatmap_homme['data'][0], row=1, col=1)

heatmap_femme = px.imshow(corr_femme, origin = 'lower')
fig.add_trace(heatmap_femme['data'][0], row=1, col=2)

fig.update_layout(height=600, width=1300, title_text='Corrélation entre les notes des partenaires et la chance d\'obtenir un second rendez-vous selon le genre')

fig.update_yaxes(autorange="reversed")

On retrouve ici des correlations importantes entre les notes données par les partenaires à propos des attributs et la chance d'obtenir un second match. Le fait de séparer les hommes et les femmes permet de mettre en évidence les différences avec le graphique précédent, sur les préférences de chacun des candidats.

La note globale semble avoir l'impact le plus important. La sincérite ou l'ambition ont des correlations plus faibles. L'intelligence semblait pourtant être au global important pour les candidats, le résultat montre une légère différence.

Le fun, l'attirance et les intérêts semblent corréler davantages pour les 2 genres. A l'inverse de l'intelligence, les intérêts semblaient ne pas avoir une forte importance pour les candidats, alors que c'est l'un des facteur les plus correlés avec le second rendez-vous.

L'attirance cependant possède une corrélation similaire entre les hommes et les femmes, alors qu'il y avait une différence significative entre les genres sur ce qu'ont annoncé les candidats initialement. A savoir que pour les hommes c'était le plus important, et pour les femmes le 3ème plus important.

In [25]:
pourcentage_match_objectif = df.groupby('objectif_c')['match'].mean().sort_values(ascending=True)*100
fig = px.bar(
       y = pourcentage_match_objectif.index,
       x = pourcentage_match_objectif.values,
       title = 'Chance d\'obtenir un second rendez-vous selon l\'objectif du candidat',
       width = 1000,
       orientation = 'h')
fig.update_layout(xaxis_title='Objectif dating', yaxis_title='Pourcentage')
fig.show()

L'objectif du candidat par rapport au speed dating semble très peu influencer le pourcentage de chance d'obtenir un second rendez-vous

In [26]:
pourcentage_match_freq = df.groupby('date_c')['match'].mean().sort_values(ascending=True)*100
fig = px.bar(
       y = pourcentage_match_freq.index,
       x = pourcentage_match_freq.values,
       title = 'Chance d\'obtenir un second rendez-vous selon la fréquence de sortie du candidat',
       width = 1000,
       orientation = 'h')
fig.update_layout(xaxis_title='Fréquence date', yaxis_title='Pourcentage')
fig.show()

Ce graphique nous montre bien que plus la fréquence de rendez-vous du candidat est élevée, plus il a de chance d'obtenir un second rendez-vous. 

Cela pourrait simplement se traduire par une plus grande expérience dans ce genre d'exercice.

In [27]:
pourcentage_choix = df.groupby('choix_c')['match'].agg(['count', 'sum'])['sum'] / df.groupby('choix_c')['match'].agg(['count', 'sum'])['count']

fig = px.bar(
       y = pourcentage_choix.index,
       x = pourcentage_choix.values,
       title = 'Chance d\'obtenir un second rendez-vous selon les choix du candidat',
       width = 1000,
       orientation = 'h')
fig.update_layout(xaxis_title='Fréquence date', yaxis_title='Pourcentage')
fig.show()

Limiter son nombre de partenaires lors de ce speed dating semble légèrement aider à l'obtention d'un second date

In [28]:
corr_candidat = df[['round_c', 'age_c', 'imprace_c', 'impreligion_c', 'satisfaction_c', 'interesse_c', 'match']].corr()

px.imshow(corr_candidat,
          title = 'Corrélation entre les infos du candidats et la chance d\'obtenir un second rendez-vous',
          width = 800,
          height = 600)

On observe des correlations assez faibles entre les variables du candidat et le nombre de second rendez-vous. 

Il semblerait qu'il y ait une correlation plus importante lorsqu'une personne pense avoir un nombre de match conséquent et son nombre réel de match.

In [29]:
corr_partenaire = df[['ordre_p', 'corr_p', 'age_p', 'match']].corr()

px.imshow(corr_partenaire,
          title = 'Corrélation entre les infos du partenaire et la chance d\'obtenir un second rendez-vous',
          width = 800,
          height = 500)

La correlation entre les données du partenaires, y compris la correlation sur les intérêts semblent faible pour estimer le second rendez-vous ou non.

Il serait intéressant de vérifier la correlation sur les attributs, et la note globale (par les partenaires)

## PARTIE 5 : Conclusion



En conclusion, l'analyse des données du speed dating révèle des tendances intéressantes, des nuances se dégagent. 

Les hommes montrent un intérêt marqué pour l'attirance physique, tandis que les femmes attribuent plus d'importance à des attributs variés, avec une légère préférence pour l'intelligence. Les corrélations entre les notes attribuées par les partenaires et les chances d'obtenir un second rendez-vous varient, la note globale étant le facteur le plus impactant. 

La fréquence des rendez-vous antérieurs semble influencer positivement les chances de succès, suggérant une possible relation avec l'expérience. Limitant le nombre de partenaires, bien que légèrement bénéfique, ne constitue pas un facteur déterminant. 

Les perceptions des candidats sur leur nombre de matchs et la réalité montrent des corrélations intéressantes, tandis que les évaluations des partenaires et les intérêts présentent des liens moins forts avec le succès d'un second rendez-vous. 

Une analyse plus approfondie des attributs et des notes globales des partenaires pourrait apporter des éclaircissements supplémentaires.